# Implement SBB + Gold portfolios with following practical restrictions
1. Cap leverage at 100%
2. Target expected return of the 1/N portfolio
3. Add risky borrowing with $r_b = r_s + 2\%$



In [1]:
import numpy as np
from cvxopt import matrix
from cvxopt.solvers import qp as Solver, options as SolverOptions
from scipy.optimize import minimize_scalar
from scipy.optimize import minimize
import plotly.graph_objects as go
from scipy.stats import multivariate_normal as mvn
import pandas as pd

import quandl
quandl.ApiConfig.api_key = "f-5zoU2G4zzHaUtkJ7BY"

## Functions

In [67]:
def tangency(means, cov, rf, short_lb):
    '''
    short_lb: lower bound on position weights
    examples: 0  = no short-selling
              -1 = no more than -100% in a given asset
              None=no restrictions on short-selling
    '''

    n = len(means)
    def f(w):
        mn = w @ means
        sd = np.sqrt(w.T @ cov @ w)
        return -(mn - rf) / sd
    # Initial guess (equal-weighted)
    w0 = (1/n)*np.ones(n)
    # Constraint: fully-invested portfolio
    A = np.ones(n)
    b = 1
    cons = [{"type": "eq", "fun": lambda x: A @ x - b}]
    bnds = [(short_lb, None) for i in range(n)] 
    # Optimization
    wgts_tangency = minimize(f, w0, bounds=bnds, constraints=cons).x
    return wgts_tangency
def gmv(cov, short_lb): 
    '''
    short_lb: lower bound on position weights
    examples: 0  = no short-selling
              -1 = no more than -100% in a given asset
              None=no restrictions on short-selling
    '''    
    n = len(cov)
    Q = matrix(cov, tc="d")
    p = matrix(np.zeros(n), (n, 1), tc="d")
    if short_lb==None:
        # No position limits
        G = matrix(np.zeros((n,n)), tc="d")
        h = matrix(np.zeros(n), (n, 1), tc="d")
    else:
        # Constraint: short-sales not allowed
        G = matrix(-np.identity(n), tc="d")
        h = matrix(-short_lb * np.ones(n), (n, 1), tc="d")
    # Fully-invested constraint
    A = matrix(np.ones(n), (1, n), tc="d")
    b = matrix([1], (1, 1), tc="d")
    sol = Solver(Q, p, G, h, A, b, options={'show_progress': False})
    wgts_gmv = np.array(sol["x"]).flatten() if sol["status"] == "optimal" else np.array(n * [np.nan])
    return wgts_gmv    

def optimal(means, cov, raver, rs, rb, short_lb):
    '''
    short_lb: lower bound on position weights
    examples: 0  = no short-selling
              -1 = no more than -100% in a given asset
              None=no restrictions on short-selling
    '''  
    n = len(cov)
    Q = np.zeros((n + 2, n + 2))
    Q[2:, 2:] = raver * cov
    Q = matrix(Q, tc="d")
    p = np.array([-rs, -rb] + list(-means))
    p = matrix(p, (n + 2, 1), tc="d")
    if short_lb==None:
        # Constraint: short-sales unconstrained, saving weight positive, borrowing weight negative
        G = np.zeros((2, n + 2))
        G[0, 0] = -1
        G[1, 1] = 1
        G = matrix(G, (2, n+2), tc="d")
        h = matrix([0, 0], (2, 1), tc="d")
    else:    
        # Constraint: short-sales constrained, saving weight positive, borrowing weight negative
        G = -np.identity(n + 2)
        G[1, 1] = 1
        G = matrix(G, (n+2, n+2), tc="d")
        h = matrix(np.zeros(n+2), (n+2, 1), tc="d")
    # Constraint: fully-invested portfolio
    A = matrix(np.ones(n+2), (1, n+2), tc="d")
    b = matrix([1], (1, 1), tc="d")
    sol = Solver(Q, p, G, h, A, b)
    # function returns vector of risky asset weights
    return np.array(sol["x"]).flatten()[2:] if sol["status"] == "optimal" else None    



## Pull the data

In [86]:
# Pull the data (from sbb.py and gold.py from website codebase)
# Stocks, bonds, bills
nominal = pd.read_csv('https://www.dropbox.com/s/hgwte6swx57jqcv/nominal_sbb.csv?dl=1', index_col=['Year'])

# Gold
d = quandl.get("LBMA/GOLD")['USD (AM)']
gold = d.resample('Y').last().iloc[:-1]
gold.index = [x.year for x in gold.index]
gold.loc[1967] = d.iloc[0]
gold = gold.sort_index().pct_change().dropna()
gold.name = 'Gold'
 

df = pd.concat((nominal, gold), axis=1).dropna()
assets = ['TBills','S&P 500', 'Gold', 'Corporates', 'Treasuries']
df = df[assets]


##### Inputs
# Window length (and initial period)
window = 20
n = len(assets)-1
raver = 5
short_lb = None
T = len(df)-window
#Extra borrowing rate (annual in decimal)
excess_rb = 0.04

## Option 0: Rolling input estimation: No constraints

In [98]:
# Refresh data
df = pd.concat((nominal, gold), axis=1).dropna()
assets = ['TBills','S&P 500', 'Gold', 'Corporates', 'Treasuries']
df = df[assets]

# Rolling input estimation
risky = assets[1:]
df.columns = ['rf']+['r'+str(i) for i in range(n)]
asset_list = [str(i) for i in range(n)]
for asset in asset_list:
    df['mn' + asset]=df['r'+asset].rolling(window).mean()
    df['sd' + asset]=df['r'+asset].rolling(window).std()

ret_list = ['r' + asset for asset in asset_list]
corrs = df[ret_list].rolling(window, min_periods=window).corr()

corr_list = []
for j, asset in enumerate(asset_list):
    for k in range(j+1,n):
        df['c'+asset+str(k)]=corrs.loc[(slice(None),'r'+asset),'r'+str(k)].values
df['year'] = df.index
df = df.reset_index()


# Prepare columns for the rolling optimization output
model_list = ['ew', 'est_all', 'est_cov', 'est_sds']
for model in model_list:
    df['portret_'+model] = np.nan      #portret is the realized portfolio return of the 100% risky asset portfolio
    if model not in ['ew']:
        for asset in asset_list:
            df['wgt' + asset + '_' +model] = np.nan
    df['wgt_cal_'+model] =np.nan
    df['raver_portret_'+model] =np.nan #raver_portret_ is the realized return of the CAL choice of the raver investor

mn_list = ['mn'+asset for asset in asset_list]
sd_list = ['sd'+asset for asset in asset_list] 

# Choose optimal portfolios each time period
for i in np.arange(window,window+T):
    # Full estimation inputs at each point in time
    means = df[mn_list].iloc[i-1].values
    sds   = df[sd_list].iloc[i-1].values
    C  = np.identity(n)
    for j, asset in enumerate(asset_list):
        for k in range(j+1,n):
            C[j, k] = C[k, j] =    df.loc[i-1,'c'+asset+str(k)]  
    cov = np.diag(sds) @ C @ np.diag(sds)

    r = df.loc[i,'rf']
    ##### Note: all portfolio weights considered to be beginning of period weights
    ##### (so multiply by contemporaneous realized returns)
    # Full estimation tangency portfolio
    model = 'est_all'
    wgts = tangency(means,cov,r,short_lb)
    for j, asset in enumerate(asset_list):
        df.loc[i,'wgt'+asset+'_' + model] = wgts[j]
    df.loc[i,'portret_'+model] = df.loc[i,ret_list].values @ wgts
    df.loc[i,'wgt_cal_'+model] = (wgts @ means - r) / (raver * (wgts @ cov @ wgts))
    df.loc[i,'wgt_cal_'+model] = max(0,df.loc[i,'wgt_cal_'+model])
    df.loc[i,'raver_portret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (df.loc[i,'portret_'+model] -r)
    df.loc[i,'expret_'+model] = means @ wgts
    df.loc[i,'sd'+model] = np.sqrt(wgts @ cov @ wgts)

    # Estimate only covariance matrix
    model = 'est_cov'
    wgts = gmv(cov,short_lb)
    for j, asset in enumerate(asset_list):
        df.loc[i,'wgt'+asset+'_' + model] = wgts[j]
    df.loc[i,'portret_'+model] = df.loc[i,ret_list].values @ wgts
    df.loc[i,'wgt_cal_'+model] = (means.mean() - r) / (raver * (wgts @ cov @ wgts))
    df.loc[i,'wgt_cal_'+model] = max(0,df.loc[i,'wgt_cal_'+model])
    df.loc[i,'raver_portret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (df.loc[i,'portret_'+model] -r)
    df.loc[i,'expret_'+model] = means.mean()
    df.loc[i,'sd'+model] = np.sqrt(wgts @ cov @ wgts)

    # Estimate only standard deviations in covariance matrix
    model = 'est_sds'
    for j, asset in enumerate(asset_list):
        for k in range(j+1,n):
            cov[j, k] = cov[k, j] = 0.0
    wgts = gmv(cov,short_lb)
    for j, asset in enumerate(asset_list):
        df.loc[i,'wgt'+asset+'_' + model] = wgts[j]
    df.loc[i,'portret_'+model] = df.loc[i,ret_list].values @ wgts
    df.loc[i,'wgt_cal_'+model] = (means.mean() - r) / (raver * (wgts @ cov @ wgts))		
    df.loc[i,'wgt_cal_'+model] = max(0,df.loc[i,'wgt_cal_'+model])
    df.loc[i,'raver_portret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (df.loc[i,'portret_'+model] -r)    
    df.loc[i,'expret_'+model] = means.mean()
    df.loc[i,'sd'+model] = np.sqrt(wgts @ cov @ wgts)

    # Equal-weighted portfolio
    model = 'ew'
    for j, asset in enumerate(asset_list):
        cov[j,j] = (sds.mean())**2
    wgts = (1/n)*np.ones(n)
    df.loc[i,'portret_'+model] = df.loc[i,ret_list].values @ wgts
    df.loc[i,'wgt_cal_'+model] = (means.mean() - r) / (raver * (wgts @ cov @ wgts))
    df.loc[i,'wgt_cal_'+model] = max(0,df.loc[i,'wgt_cal_'+model])
    df.loc[i,'raver_portret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (df.loc[i,'portret_'+model] -r)   
    df.loc[i,'expret_'+model] = means.mean()
    df.loc[i,'sd'+model] = np.sqrt(wgts @ cov @ wgts)    

In [35]:
portret_list = ['raver_portret_' +  model for model in ['est_all', 'est_cov', 'est_sds','ew']]
stats = df[portret_list].describe()
stats

,raver_portret_est_all,raver_portret_est_cov,raver_portret_est_sds,raver_portret_ew
count,34.000000,34.000000,34.000000,34.000000
mean,0.298606,0.305746,0.317689,0.212242
std,0.406017,0.371266,0.331448,0.242597
min,-0.802216,-0.454778,-0.302823,-0.257604
25%,0.069471,0.135104,0.036213,0.035269
50%,0.351288,0.241430,0.347497,0.212161
75%,0.568270,0.453158,0.543779,0.309358
max,1.176866,1.045235,0.974309,0.701113


In [36]:
# Summarize sharpe ratio, avg ret, sd(ret) for each model
sr_df = pd.DataFrame(dtype=float, columns = ['sr','avg_ret','sd_ret'], index = ['est_all', 'est_cov', 'est_sds','ew'])
r = df[np.isnan(df['raver_portret_ew'])==False].rf.mean()
for model in ['est_all', 'est_cov', 'est_sds','ew']:
    sr_df.loc[model,'sr'] = (stats.loc['mean','raver_portret_' +  model] - r)/stats.loc['std','raver_portret_' +  model]
    sr_df.loc[model,'avg_ret'] = stats.loc['mean','raver_portret_' +  model]
    sr_df.loc[model,'sd_ret'] = stats.loc['std','raver_portret_' +  model]

label_dict = {'true': 'theoretical optimal weights', 
            'ew': 'equal weights',
            'est_all': 'estimate all inputs',
            'est_cov': 'estimate covariance matrix only',
            'est_sds': 'estimate standard deviations only'}

xaxis_label_dict = {'true': 'theoretical optimal weights', 
            'ew': 'Est-None',
            'est_all': 'Est-All',
            'est_cov': 'Est-SD-Corr',
            'est_sds': 'Est-SD'}
sr_df = sr_df.reset_index()
sr_df['label'] = sr_df['index'].apply(lambda x: label_dict[x])
sr_df['xaxis_label'] = sr_df['index'].apply(lambda x: xaxis_label_dict[x])

In [37]:
df[np.isnan(df['raver_portret_ew'])==False].rf
sr_df

,index,sr,avg_ret,sd_ret,label,xaxis_label
0,est_all,0.666248,0.298606,0.406017,estimate all inputs,Est-All
1,est_cov,0.747840,0.305746,0.371266,estimate covariance matrix only,Est-SD-Corr
2,est_sds,0.873715,0.317689,0.331448,estimate standard deviations only,Est-SD
3,ew,0.759052,0.212242,0.242597,equal weights,Est-None


In [38]:
import plotly.graph_objects as go

string ="Strategy: %{customdata[0]} <br>"
string += "Sharpe ratio: %{y:0.3f}<br>"
string += "Average return: %{customdata[1]:0.1%}<br>"
string += "SD(return): %{customdata[2]:0.1%}<br>"
string += "<extra></extra>"

fig = go.Figure()
fig.add_trace(go.Bar(x=sr_df['xaxis_label'], y=sr_df['sr'], customdata=sr_df[['label','avg_ret','sd_ret']], hovertemplate=string))
fig.layout.yaxis["title"] = "Sharpe ratio"
fig.layout.xaxis["title"] = "Strategy"
fig.show()

In [11]:
# Plot the time-series of returns and portfolio weights.
for asset in asset_list:
    df['wgt'+asset + '_ew'] = 1/n
label_dict = {'true': 'theoretical optimal weights', 
            'ew': 'equal weights',
            'est_all': 'estimate all inputs',
            'est_cov': 'estimate covariance matrix only',
            'est_sds': 'estimate standard deviations only'}

xaxis_label_dict = {'true': 'theoretical optimal weights', 
            'ew': 'Est-None',
            'est_all': 'Est-All',
            'est_cov': 'Est-SD-Corr',
            'est_sds': 'Est-SD'}           

fig = go.Figure()
for model in ['est_all', 'est_cov', 'est_sds', 'ew']:
    string =  "Strategy: " + label_dict[model] +" <br>"
    string += "Year: %{x:4.0f}<br>"
    string += "Return: %{y:0.1%}<br>"
    string += "Weight in Risky Portfolio: %{customdata[0]: 0.1%} <br>"
    string += "Risky Portfolio Weights:<br>"
    string += "  "+ risky[0] +": %{customdata[1]: 0.1%} <br>"
    string += "  "+ risky[1] +": %{customdata[2]: 0.1%} <br>"
    string += "  "+ risky[2] +": %{customdata[3]: 0.1%} <br>"
    string += "  "+ risky[3] +": %{customdata[4]: 0.1%} <br>"
    string += "<extra></extra>"

    wgt_list = ['wgt_cal_'+ model] + ['wgt'+asset + "_" + model for asset in asset_list]
    trace=go.Scatter(x=df['year'], y=df['raver_portret_'+model], customdata=df[wgt_list], hovertemplate=string, name = xaxis_label_dict[model])
    fig.add_trace(trace)
fig.layout.yaxis["title"] = "Return"
fig.layout.xaxis["title"] = "Year"
fig.update_yaxes(tickformat=".0%")
fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.01))
fig.update_xaxes(range=[df['year'].iloc[window], df.year.max()])
fig.show()

In [12]:
# Plot leverage (wgt_cal)
fig = go.Figure()
for model in ['est_all', 'est_cov', 'est_sds', 'ew']:
    string =  "Strategy: " + label_dict[model] +" <br>"
    string += "Year: %{x:4.0f}<br>"
    string += "Weight in Risky Portfolio: %{y: 0.1%} <br>"
    string += "<extra></extra>"

    trace=go.Scatter(x=df['year'], y=df['wgt_cal_'+model], hovertemplate=string, name = xaxis_label_dict[model])
    fig.add_trace(trace)
fig.layout.yaxis["title"] = "Risky Portfolio Weight"
fig.layout.xaxis["title"] = "Year"
fig.update_yaxes(tickformat=".0%")
fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.01))
fig.update_xaxes(range=[df['year'].iloc[window], df.year.max()])
fig.show()

In [99]:
wgt_list = ['wgt_cal_' +  model for model in ['est_all', 'est_cov', 'est_sds','ew']]
stats = df[wgt_list].describe()
stats

,wgt_cal_est_all,wgt_cal_est_cov,wgt_cal_est_sds,wgt_cal_ew
count,34.000000,34.000000,34.000000,34.000000
mean,4.543357,4.541040,4.954805,3.049132
std,1.581558,2.138640,1.680892,1.268526
min,1.012865,1.173961,1.668419,0.755461
25%,3.742845,2.990809,3.629274,1.876814
50%,4.291620,4.050710,4.915981,3.295829
75%,5.632940,5.913397,6.301853,4.286413
max,7.515303,10.201208,7.674379,4.859606


## Option 1: Rolling input estimation: Cap leverage at 100%

In [100]:
# Refresh data
df = pd.concat((nominal, gold), axis=1).dropna()
assets = ['TBills','S&P 500', 'Gold', 'Corporates', 'Treasuries']
df = df[assets]

# Rolling input estimation
risky = assets[1:]
df.columns = ['rf']+['r'+str(i) for i in range(n)]
asset_list = [str(i) for i in range(n)]
for asset in asset_list:
    df['mn' + asset]=df['r'+asset].rolling(window).mean()
    df['sd' + asset]=df['r'+asset].rolling(window).std()

ret_list = ['r' + asset for asset in asset_list]
corrs = df[ret_list].rolling(window, min_periods=window).corr()

corr_list = []
for j, asset in enumerate(asset_list):
    for k in range(j+1,n):
        df['c'+asset+str(k)]=corrs.loc[(slice(None),'r'+asset),'r'+str(k)].values
df['year'] = df.index
df = df.reset_index()


# Prepare columns for the rolling optimization output
model_list = ['ew', 'est_all', 'est_cov', 'est_sds']
for model in model_list:
    df['portret_'+model] = np.nan      #portret is the realized portfolio return of the 100% risky asset portfolio
    if model not in ['ew']:
        for asset in asset_list:
            df['wgt' + asset + '_' +model] = np.nan
    df['wgt_cal_'+model] =np.nan
    df['raver_portret_'+model] =np.nan #raver_portret_ is the realized return of the CAL choice of the raver investor

mn_list = ['mn'+asset for asset in asset_list]
sd_list = ['sd'+asset for asset in asset_list] 

In [101]:
max_leverage = 2.0

# Choose optimal portfolios each time period
for i in np.arange(window,window+T):
    # Full estimation inputs at each point in time
    means = df[mn_list].iloc[i-1].values
    sds   = df[sd_list].iloc[i-1].values
    C  = np.identity(n)
    for j, asset in enumerate(asset_list):
        for k in range(j+1,n):
            C[j, k] = C[k, j] =    df.loc[i-1,'c'+asset+str(k)]  
    cov = np.diag(sds) @ C @ np.diag(sds)

    r = df.loc[i,'rf']
    ##### Note: all portfolio weights considered to be beginning of period weights
    ##### (so multiply by contemporaneous realized returns)
    # Full estimation tangency portfolio
    model = 'est_all'
    wgts = tangency(means,cov,r,short_lb)
    for j, asset in enumerate(asset_list):
        df.loc[i,'wgt'+asset+'_' + model] = wgts[j]
    df.loc[i,'portret_'+model] = df.loc[i,ret_list].values @ wgts
    df.loc[i,'wgt_cal_'+model] = (wgts @ means - r) / (raver * (wgts @ cov @ wgts))
    df.loc[i,'wgt_cal_'+model] = max(0,df.loc[i,'wgt_cal_'+model])
    df.loc[i,'wgt_cal_'+model] = min(max_leverage,df.loc[i,'wgt_cal_'+model]) 
    df.loc[i,'raver_portret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (df.loc[i,'portret_'+model] -r)
    df.loc[i,'expret_'+model] = means @ wgts
    df.loc[i,'sd'+model] = np.sqrt(wgts @ cov @ wgts)

    # Estimate only covariance matrix
    model = 'est_cov'
    wgts = gmv(cov,short_lb)
    for j, asset in enumerate(asset_list):
        df.loc[i,'wgt'+asset+'_' + model] = wgts[j]
    df.loc[i,'portret_'+model] = df.loc[i,ret_list].values @ wgts
    df.loc[i,'wgt_cal_'+model] = (means.mean() - r) / (raver * (wgts @ cov @ wgts))
    df.loc[i,'wgt_cal_'+model] = max(0,df.loc[i,'wgt_cal_'+model])
    df.loc[i,'wgt_cal_'+model] = min(max_leverage,df.loc[i,'wgt_cal_'+model]) 
    df.loc[i,'raver_portret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (df.loc[i,'portret_'+model] -r)
    df.loc[i,'expret_'+model] = means.mean()
    df.loc[i,'sd'+model] = np.sqrt(wgts @ cov @ wgts)

    # Estimate only standard deviations in covariance matrix
    model = 'est_sds'
    for j, asset in enumerate(asset_list):
        for k in range(j+1,n):
            cov[j, k] = cov[k, j] = 0.0
    wgts = gmv(cov,short_lb)
    for j, asset in enumerate(asset_list):
        df.loc[i,'wgt'+asset+'_' + model] = wgts[j]
    df.loc[i,'portret_'+model] = df.loc[i,ret_list].values @ wgts
    df.loc[i,'wgt_cal_'+model] = (means.mean() - r) / (raver * (wgts @ cov @ wgts))		
    df.loc[i,'wgt_cal_'+model] = max(0,df.loc[i,'wgt_cal_'+model])
    df.loc[i,'wgt_cal_'+model] = min(max_leverage,df.loc[i,'wgt_cal_'+model]) 
    df.loc[i,'raver_portret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (df.loc[i,'portret_'+model] -r)    
    df.loc[i,'expret_'+model] = means.mean()
    df.loc[i,'sd'+model] = np.sqrt(wgts @ cov @ wgts)

    # Equal-weighted portfolio
    model = 'ew'
    for j, asset in enumerate(asset_list):
        cov[j,j] = (sds.mean())**2
    wgts = (1/n)*np.ones(n)
    df.loc[i,'portret_'+model] = df.loc[i,ret_list].values @ wgts
    df.loc[i,'wgt_cal_'+model] = (means.mean() - r) / (raver * (wgts @ cov @ wgts))
    df.loc[i,'wgt_cal_'+model] = max(0,df.loc[i,'wgt_cal_'+model])
    df.loc[i,'wgt_cal_'+model] = min(max_leverage,df.loc[i,'wgt_cal_'+model]) 
    df.loc[i,'raver_portret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (df.loc[i,'portret_'+model] -r)   
    df.loc[i,'expret_'+model] = means.mean()
    df.loc[i,'sd'+model] = np.sqrt(wgts @ cov @ wgts)    

In [41]:
portret_list = ['raver_portret_' +  model for model in ['est_all', 'est_cov', 'est_sds','ew']]
stats = df[portret_list].describe()
stats

,raver_portret_est_all,raver_portret_est_cov,raver_portret_est_sds,raver_portret_ew
count,34.000000,34.000000,34.000000,34.000000
mean,0.137336,0.141484,0.142960,0.132050
std,0.162242,0.133273,0.126154,0.126848
min,-0.249975,-0.143157,-0.111202,-0.104199
25%,0.049994,0.058107,0.036562,0.035269
50%,0.158966,0.156445,0.162290,0.137533
75%,0.264941,0.223857,0.243130,0.210632
max,0.435923,0.424768,0.426391,0.359540


In [48]:
sr_df = pd.DataFrame(dtype=float, columns = ['sr','avg_ret','sd_ret'], index = ['est_all', 'est_cov', 'est_sds','ew'])
r = df[np.isnan(df['raver_portret_ew'])==False].rf.mean()
for model in ['est_all', 'est_cov', 'est_sds','ew']:
    sr_df.loc[model,'sr'] = (stats.loc['mean','raver_portret_' +  model] - r)/stats.loc['std','raver_portret_' +  model]
    sr_df.loc[model,'avg_ret'] = stats.loc['mean','raver_portret_' +  model]
    sr_df.loc[model,'sd_ret'] = stats.loc['std','raver_portret_' +  model]

label_dict = {'true': 'theoretical optimal weights', 
            'ew': 'equal weights',
            'est_all': 'estimate all inputs',
            'est_cov': 'estimate covariance matrix only',
            'est_sds': 'estimate standard deviations only'}

xaxis_label_dict = {'true': 'theoretical optimal weights', 
            'ew': 'Est-None',
            'est_all': 'Est-All',
            'est_cov': 'Est-SD-Corr',
            'est_sds': 'Est-SD'}
sr_df = sr_df.reset_index()
sr_df['label'] = sr_df['index'].apply(lambda x: label_dict[x])
sr_df['xaxis_label'] = sr_df['index'].apply(lambda x: xaxis_label_dict[x])

In [43]:
df[np.isnan(df['raver_portret_ew'])==False].rf
sr_df

,index,sr,avg_ret,sd_ret,label,xaxis_label
0,est_all,0.673303,0.137336,0.162242,estimate all inputs,Est-All
1,est_cov,0.850779,0.141484,0.133273,estimate covariance matrix only,Est-SD-Corr
2,est_sds,0.910491,0.142960,0.126154,estimate standard deviations only,Est-SD
3,ew,0.819499,0.132050,0.126848,equal weights,Est-None


In [44]:
import plotly.graph_objects as go

string ="Strategy: %{customdata[0]} <br>"
string += "Sharpe ratio: %{y:0.3f}<br>"
string += "Average return: %{customdata[1]:0.1%}<br>"
string += "SD(return): %{customdata[2]:0.1%}<br>"
string += "<extra></extra>"

fig = go.Figure()
fig.add_trace(go.Bar(x=sr_df['xaxis_label'], y=sr_df['sr'], customdata=sr_df[['label','avg_ret','sd_ret']], hovertemplate=string))
fig.layout.yaxis["title"] = "Sharpe ratio"
fig.layout.xaxis["title"] = "Strategy"
fig.show()

In [45]:
# Plot the time-series of returns and portfolio weights.
for asset in asset_list:
    df['wgt'+asset + '_ew'] = 1/n
label_dict = {'true': 'theoretical optimal weights', 
            'ew': 'equal weights',
            'est_all': 'estimate all inputs',
            'est_cov': 'estimate covariance matrix only',
            'est_sds': 'estimate standard deviations only'}

xaxis_label_dict = {'true': 'theoretical optimal weights', 
            'ew': 'Est-None',
            'est_all': 'Est-All',
            'est_cov': 'Est-SD-Corr',
            'est_sds': 'Est-SD'}           

fig = go.Figure()
for model in ['est_all', 'est_cov', 'est_sds', 'ew']:
    string =  "Strategy: " + label_dict[model] +" <br>"
    string += "Year: %{x:4.0f}<br>"
    string += "Return: %{y:0.1%}<br>"
    string += "Weight in Risky Portfolio: %{customdata[0]: 0.1%} <br>"
    string += "Risky Portfolio Weights:<br>"
    string += "  "+ risky[0] +": %{customdata[1]: 0.1%} <br>"
    string += "  "+ risky[1] +": %{customdata[2]: 0.1%} <br>"
    string += "  "+ risky[2] +": %{customdata[3]: 0.1%} <br>"
    string += "  "+ risky[3] +": %{customdata[4]: 0.1%} <br>"
    string += "<extra></extra>"

    wgt_list = ['wgt_cal_'+ model] + ['wgt'+asset + "_" + model for asset in asset_list]
    trace=go.Scatter(x=df['year'], y=df['raver_portret_'+model], customdata=df[wgt_list], hovertemplate=string, name = xaxis_label_dict[model])
    fig.add_trace(trace)
fig.layout.yaxis["title"] = "Return"
fig.layout.xaxis["title"] = "Year"
fig.update_yaxes(tickformat=".0%")
fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.01))
fig.update_xaxes(range=[df['year'].iloc[window], df.year.max()])
fig.show()

In [46]:
# Plot leverage (wgt_cal)
fig = go.Figure()
for model in ['est_all', 'est_cov', 'est_sds', 'ew']:
    string =  "Strategy: " + label_dict[model] +" <br>"
    string += "Year: %{x:4.0f}<br>"
    string += "Weight in Risky Portfolio: %{y: 0.1%} <br>"
    string += "<extra></extra>"

    trace=go.Scatter(x=df['year'], y=df['wgt_cal_'+model], hovertemplate=string, name = xaxis_label_dict[model])
    fig.add_trace(trace)
fig.layout.yaxis["title"] = "Risky Portfolio Weight"
fig.layout.xaxis["title"] = "Year"
fig.update_yaxes(tickformat=".0%")
fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.01))
fig.update_xaxes(range=[df['year'].iloc[window], df.year.max()])
fig.show()

In [47]:
# Plot expected return (expret)
fig = go.Figure()
for model in ['est_all', 'est_cov', 'est_sds', 'ew']:
    string =  "Strategy: " + label_dict[model] +" <br>"
    string += "Year: %{x:4.0f}<br>"
    string += "Expected Return: %{y: 0.1%} <br>"
    string += "<extra></extra>"

    trace=go.Scatter(x=df['year'], y=df['expret_'+model], hovertemplate=string, name = xaxis_label_dict[model])
    fig.add_trace(trace)
fig.layout.yaxis["title"] = "Expected Return"
fig.layout.xaxis["title"] = "Year"
fig.update_yaxes(tickformat=".0%")
fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.01))
fig.update_xaxes(range=[df['year'].iloc[window], df.year.max()])
fig.show()

In [102]:
wgt_list = ['wgt_cal_' +  model for model in ['est_all', 'est_cov', 'est_sds','ew']]
stats = df[wgt_list].describe()
stats

,wgt_cal_est_all,wgt_cal_est_cov,wgt_cal_est_sds,wgt_cal_ew
count,34.000000,34.000000,34.000000,34.000000
mean,1.953613,1.961716,1.990248,1.860352
std,0.194579,0.151018,0.056866,0.295111
min,1.012865,1.173961,1.668419,0.755461
25%,2.000000,2.000000,2.000000,1.876814
50%,2.000000,2.000000,2.000000,2.000000
75%,2.000000,2.000000,2.000000,2.000000
max,2.000000,2.000000,2.000000,2.000000


# Option 2: Match 1/N expected return

In [103]:
# Refresh data
df = pd.concat((nominal, gold), axis=1).dropna()
assets = ['TBills','S&P 500', 'Gold', 'Corporates', 'Treasuries']
df = df[assets]

# Rolling input estimation
risky = assets[1:]
df.columns = ['rf']+['r'+str(i) for i in range(n)]
asset_list = [str(i) for i in range(n)]
for asset in asset_list:
    df['mn' + asset]=df['r'+asset].rolling(window).mean()
    df['sd' + asset]=df['r'+asset].rolling(window).std()

ret_list = ['r' + asset for asset in asset_list]
corrs = df[ret_list].rolling(window, min_periods=window).corr()

corr_list = []
for j, asset in enumerate(asset_list):
    for k in range(j+1,n):
        df['c'+asset+str(k)]=corrs.loc[(slice(None),'r'+asset),'r'+str(k)].values
df['year'] = df.index
df = df.reset_index()


# Prepare columns for the rolling optimization output
model_list = ['ew', 'est_all', 'est_cov', 'est_sds']
for model in model_list:
    df['portret_'+model] = np.nan      #portret is the realized portfolio return of the 100% risky asset portfolio
    if model not in ['ew']:
        for asset in asset_list:
            df['wgt' + asset + '_' +model] = np.nan
    df['wgt_cal_'+model] =np.nan
    df['raver_portret_'+model] =np.nan #raver_portret_ is the realized return of the CAL choice of the raver investor

mn_list = ['mn'+asset for asset in asset_list]
sd_list = ['sd'+asset for asset in asset_list] 

# Choose optimal portfolios each time period
for i in np.arange(window,window+T):
    # Full estimation inputs at each point in time
    means = df[mn_list].iloc[i-1].values
    sds   = df[sd_list].iloc[i-1].values
    C  = np.identity(n)
    for j, asset in enumerate(asset_list):
        for k in range(j+1,n):
            C[j, k] = C[k, j] =    df.loc[i-1,'c'+asset+str(k)]  
    cov = np.diag(sds) @ C @ np.diag(sds)

    r = df.loc[i,'rf']
    ##### Note: all portfolio weights considered to be beginning of period weights
    ##### (so multiply by contemporaneous realized returns)
    # Full estimation tangency portfolio
    model = 'est_all'
    wgts = tangency(means,cov,r,short_lb)
    for j, asset in enumerate(asset_list):
        df.loc[i,'wgt'+asset+'_' + model] = wgts[j]
    df.loc[i,'portret_'+model] = df.loc[i,ret_list].values @ wgts
    # df.loc[i,'wgt_cal_'+model] = (wgts @ means - r) / (raver * (wgts @ cov @ wgts))
    # df.loc[i,'wgt_cal_'+model] = max(0,df.loc[i,'wgt_cal_'+model])
    df.loc[i,'wgt_cal_'+model] = (means.mean() - r)/ (wgts @ means - r)
    df.loc[i,'raver_portret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (df.loc[i,'portret_'+model] -r)
    df.loc[i,'expret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (means @ wgts - r)
    df.loc[i,'sd'+model] = df.loc[i,'wgt_cal_'+model] * np.sqrt(wgts @ cov @ wgts)

    # Estimate only covariance matrix
    model = 'est_cov'
    wgts = gmv(cov,short_lb)
    for j, asset in enumerate(asset_list):
        df.loc[i,'wgt'+asset+'_' + model] = wgts[j]
    df.loc[i,'portret_'+model] = df.loc[i,ret_list].values @ wgts
    # df.loc[i,'wgt_cal_'+model] = (means.mean() - r) / (raver * (wgts @ cov @ wgts))
    # df.loc[i,'wgt_cal_'+model] = max(0,df.loc[i,'wgt_cal_'+model])
    df.loc[i,'wgt_cal_'+model] = 1.0
    df.loc[i,'raver_portret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (df.loc[i,'portret_'+model] -r)
    df.loc[i,'expret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (means.mean() - r)
    df.loc[i,'sd'+model] = df.loc[i,'wgt_cal_'+model] *np.sqrt(wgts @ cov @ wgts)

    # Estimate only standard deviations in covariance matrix
    model = 'est_sds'
    for j, asset in enumerate(asset_list):
        for k in range(j+1,n):
            cov[j, k] = cov[k, j] = 0.0
    wgts = gmv(cov,short_lb)
    for j, asset in enumerate(asset_list):
        df.loc[i,'wgt'+asset+'_' + model] = wgts[j]
    df.loc[i,'portret_'+model] = df.loc[i,ret_list].values @ wgts
    # df.loc[i,'wgt_cal_'+model] = (means.mean() - r) / (raver * (wgts @ cov @ wgts))		
    # df.loc[i,'wgt_cal_'+model] = max(0,df.loc[i,'wgt_cal_'+model])
    df.loc[i,'wgt_cal_'+model] = 1.0
    df.loc[i,'raver_portret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (df.loc[i,'portret_'+model] -r)    
    df.loc[i,'expret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (means.mean() - r)
    df.loc[i,'sd'+model] = df.loc[i,'wgt_cal_'+model] *np.sqrt(wgts @ cov @ wgts)

    # Equal-weighted portfolio
    model = 'ew'
    for j, asset in enumerate(asset_list):
        cov[j,j] = (sds.mean())**2
    wgts = (1/n)*np.ones(n)
    df.loc[i,'portret_'+model] = df.loc[i,ret_list].values @ wgts
    # df.loc[i,'wgt_cal_'+model] = (means.mean() - r) / (raver * (wgts @ cov @ wgts))
    # df.loc[i,'wgt_cal_'+model] = max(0,df.loc[i,'wgt_cal_'+model])
    df.loc[i,'wgt_cal_'+model] = 1.0
    df.loc[i,'raver_portret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (df.loc[i,'portret_'+model] -r)   
    df.loc[i,'expret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (means.mean() - r)
    df.loc[i,'sd'+model] = df.loc[i,'wgt_cal_'+model] *np.sqrt(wgts @ cov @ wgts)    

In [58]:
portret_list = ['raver_portret_' +  model for model in ['est_all', 'est_cov', 'est_sds','ew']]
stats = df[portret_list].describe()
stats

,raver_portret_est_all,raver_portret_est_cov,raver_portret_est_sds,raver_portret_ew
count,34.000000,34.000000,34.000000,34.000000
mean,0.076624,0.085547,0.086030,0.082503
std,0.065100,0.067320,0.065648,0.066292
min,-0.101785,-0.064754,-0.042702,-0.045274
25%,0.040639,0.047370,0.047921,0.040752
50%,0.087196,0.099889,0.086447,0.081351
75%,0.127263,0.123685,0.127238,0.131893
max,0.189141,0.217438,0.240645,0.204826


In [59]:
# Summarize sharpe ratio, avg ret, sd(ret) for each model
sr_df = pd.DataFrame(dtype=float, columns = ['sr','avg_ret','sd_ret'], index = ['est_all', 'est_cov', 'est_sds','ew'])
r = df[np.isnan(df['raver_portret_ew'])==False].rf.mean()
for model in ['est_all', 'est_cov', 'est_sds','ew']:
    sr_df.loc[model,'sr'] = (stats.loc['mean','raver_portret_' +  model] - r)/stats.loc['std','raver_portret_' +  model]
    sr_df.loc[model,'avg_ret'] = stats.loc['mean','raver_portret_' +  model]
    sr_df.loc[model,'sd_ret'] = stats.loc['std','raver_portret_' +  model]

label_dict = {'true': 'theoretical optimal weights', 
            'ew': 'equal weights',
            'est_all': 'estimate all inputs',
            'est_cov': 'estimate covariance matrix only',
            'est_sds': 'estimate standard deviations only'}

xaxis_label_dict = {'true': 'theoretical optimal weights', 
            'ew': 'Est-None',
            'est_all': 'Est-All',
            'est_cov': 'Est-SD-Corr',
            'est_sds': 'Est-SD'}
sr_df = sr_df.reset_index()
sr_df['label'] = sr_df['index'].apply(lambda x: label_dict[x])
sr_df['xaxis_label'] = sr_df['index'].apply(lambda x: xaxis_label_dict[x])

df[np.isnan(df['raver_portret_ew'])==False].rf
sr_df

,index,sr,avg_ret,sd_ret,label,xaxis_label
0,est_all,0.745419,0.076624,0.065100,estimate all inputs,Est-All
1,est_cov,0.853379,0.085547,0.067320,estimate covariance matrix only,Est-SD-Corr
2,est_sds,0.882473,0.086030,0.065648,estimate standard deviations only,Est-SD
3,ew,0.820695,0.082503,0.066292,equal weights,Est-None


In [60]:
import plotly.graph_objects as go

string ="Strategy: %{customdata[0]} <br>"
string += "Sharpe ratio: %{y:0.3f}<br>"
string += "Average return: %{customdata[1]:0.1%}<br>"
string += "SD(return): %{customdata[2]:0.1%}<br>"
string += "<extra></extra>"

fig = go.Figure()
fig.add_trace(go.Bar(x=sr_df['xaxis_label'], y=sr_df['sr'], customdata=sr_df[['label','avg_ret','sd_ret']], hovertemplate=string))
fig.layout.yaxis["title"] = "Sharpe ratio"
fig.layout.xaxis["title"] = "Strategy"
fig.show()

In [61]:
# Plot the time-series of returns and portfolio weights.
for asset in asset_list:
    df['wgt'+asset + '_ew'] = 1/n
label_dict = {'true': 'theoretical optimal weights', 
            'ew': 'equal weights',
            'est_all': 'estimate all inputs',
            'est_cov': 'estimate covariance matrix only',
            'est_sds': 'estimate standard deviations only'}

xaxis_label_dict = {'true': 'theoretical optimal weights', 
            'ew': 'Est-None',
            'est_all': 'Est-All',
            'est_cov': 'Est-SD-Corr',
            'est_sds': 'Est-SD'}           

fig = go.Figure()
for model in ['est_all', 'est_cov', 'est_sds', 'ew']:
    string =  "Strategy: " + label_dict[model] +" <br>"
    string += "Year: %{x:4.0f}<br>"
    string += "Return: %{y:0.1%}<br>"
    string += "Weight in Risky Portfolio: %{customdata[0]: 0.1%} <br>"
    string += "Risky Portfolio Weights:<br>"
    string += "  "+ risky[0] +": %{customdata[1]: 0.1%} <br>"
    string += "  "+ risky[1] +": %{customdata[2]: 0.1%} <br>"
    string += "  "+ risky[2] +": %{customdata[3]: 0.1%} <br>"
    string += "  "+ risky[3] +": %{customdata[4]: 0.1%} <br>"
    string += "<extra></extra>"

    wgt_list = ['wgt_cal_'+ model] + ['wgt'+asset + "_" + model for asset in asset_list]
    trace=go.Scatter(x=df['year'], y=df['raver_portret_'+model], customdata=df[wgt_list], hovertemplate=string, name = xaxis_label_dict[model])
    fig.add_trace(trace)
fig.layout.yaxis["title"] = "Return"
fig.layout.xaxis["title"] = "Year"
fig.update_yaxes(tickformat=".0%")
fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.01))
fig.update_xaxes(range=[df['year'].iloc[window], df.year.max()])
fig.show()

In [62]:
# Plot leverage (wgt_cal)
fig = go.Figure()
for model in ['est_all', 'est_cov', 'est_sds', 'ew']:
    string =  "Strategy: " + label_dict[model] +" <br>"
    string += "Year: %{x:4.0f}<br>"
    string += "Weight in Risky Portfolio: %{y: 0.1%} <br>"
    string += "<extra></extra>"

    trace=go.Scatter(x=df['year'], y=df['wgt_cal_'+model], hovertemplate=string, name = xaxis_label_dict[model])
    fig.add_trace(trace)
fig.layout.yaxis["title"] = "Risky Portfolio Weight"
fig.layout.xaxis["title"] = "Year"
fig.update_yaxes(tickformat=".0%")
fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.01))
fig.update_xaxes(range=[df['year'].iloc[window], df.year.max()])
fig.show()

In [63]:
# Plot expected return (expret_)
fig = go.Figure()
for model in ['est_all', 'est_cov', 'est_sds', 'ew']:
    string =  "Strategy: " + label_dict[model] +" <br>"
    string += "Year: %{x:4.0f}<br>"
    string += "Expected Return: %{y: 0.1%} <br>"
    string += "<extra></extra>"

    trace=go.Scatter(x=df['year'], y=df['expret_'+model], hovertemplate=string, name = xaxis_label_dict[model])
    fig.add_trace(trace)
fig.layout.yaxis["title"] = "Expected Return"
fig.layout.xaxis["title"] = "Year"
fig.update_yaxes(tickformat=".0%")
fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.01))
fig.update_xaxes(range=[df['year'].iloc[window], df.year.max()])
fig.show()

In [64]:
# Plot standard deviation (sd)
fig = go.Figure()
for model in ['est_all', 'est_cov', 'est_sds', 'ew']:
    string =  "Strategy: " + label_dict[model] +" <br>"
    string += "Year: %{x:4.0f}<br>"
    string += "Expected Standard Deviation: %{y: 0.1%} <br>"
    string += "<extra></extra>"

    trace=go.Scatter(x=df['year'], y=df['sd'+model], hovertemplate=string, name = xaxis_label_dict[model])
    fig.add_trace(trace)
fig.layout.yaxis["title"] = "Standard Deviation (Expected)"
fig.layout.xaxis["title"] = "Year"
fig.update_yaxes(tickformat=".0%")
fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.01))
fig.update_xaxes(range=[df['year'].iloc[window], df.year.max()])
fig.show()

In [104]:
wgt_list = ['wgt_cal_' +  model for model in ['est_all', 'est_cov', 'est_sds','ew']]
stats = df[wgt_list].describe()
stats

,wgt_cal_est_all,wgt_cal_est_cov,wgt_cal_est_sds,wgt_cal_ew
count,34.000000,34.0,34.0,34.0
mean,0.805058,1.0,1.0,1.0
std,0.258204,0.0,0.0,0.0
min,0.236844,1.0,1.0,1.0
25%,0.602928,1.0,1.0,1.0
50%,0.808654,1.0,1.0,1.0
75%,1.031691,1.0,1.0,1.0
max,1.267076,1.0,1.0,1.0


## Option 3: Different risk-free borrowing and lending rates

In [ ]:
# Refresh data
df = pd.concat((nominal, gold), axis=1).dropna()
assets = ['TBills','S&P 500', 'Gold', 'Corporates', 'Treasuries']
df = df[assets]

# Rolling input estimation
risky = assets[1:]
df.columns = ['rf']+['r'+str(i) for i in range(n)]
asset_list = [str(i) for i in range(n)]
for asset in asset_list:
    df['mn' + asset]=df['r'+asset].rolling(window).mean()
    df['sd' + asset]=df['r'+asset].rolling(window).std()

ret_list = ['r' + asset for asset in asset_list]
corrs = df[ret_list].rolling(window, min_periods=window).corr()

corr_list = []
for j, asset in enumerate(asset_list):
    for k in range(j+1,n):
        df['c'+asset+str(k)]=corrs.loc[(slice(None),'r'+asset),'r'+str(k)].values
df['year'] = df.index
df = df.reset_index()


# Prepare columns for the rolling optimization output
model_list = ['ew', 'est_all', 'est_cov', 'est_sds']
for model in model_list:
    # df['portret_'+model] = np.nan      #portret is the realized portfolio return of the 100% risky asset portfolio
    # if model not in ['ew']:
    for asset in asset_list:
        df['wgt' + asset + '_' +model] = np.nan
    df['wgt_cal_'+model] =np.nan
    df['raver_portret_'+model] =np.nan   #raver_portret_ is the realized return of the CAL choice of the raver investor

mn_list = ['mn'+asset for asset in asset_list]
sd_list = ['sd'+asset for asset in asset_list] 

# Choose optimal portfolios each time period
for i in np.arange(window,window+T):
    # Full estimation inputs at each point in time
    means = df[mn_list].iloc[i-1].values
    sds   = df[sd_list].iloc[i-1].values
    C  = np.identity(n)
    for j, asset in enumerate(asset_list):
        for k in range(j+1,n):
            C[j, k] = C[k, j] =    df.loc[i-1,'c'+asset+str(k)]  
    cov = np.diag(sds) @ C @ np.diag(sds)

    r = df.loc[i,'rf']
    rb = r + excess_rb
    ##### Note: all portfolio weights considered to be beginning of period weights
    ##### (so multiply by contemporaneous realized returns)
    # Full estimation tangency portfolio
    model = 'est_all'
    wgts = optimal(means, cov, raver, r, rb, short_lb)
    for j, asset in enumerate(asset_list):
        df.loc[i,'wgt'+asset+'_' + model] = wgts[j]
    df.loc[i,'wgt_cal_'+model] = np.sum(wgts)
    df.loc[i,'raver_portret_'+model] = r + wgts @ (df.loc[i,ret_list].values - r ) if np.sum(wgts)<=1 else rb + wgts @ (df.loc[i,ret_list].values -rb)
    df.loc[i,'expret_'+model] = r + wgts @ (means-r) if np.sum(wgts)<=1 else rb + wgts @ (means-rb)
    df.loc[i,'sd'+model] = np.sqrt(wgts @ cov @ wgts)

    # Estimate only covariance matrix
    model = 'est_cov'
    means = means.mean() * np.ones(n)
    wgts = optimal(means, cov, raver, r, rb, short_lb)
    for j, asset in enumerate(asset_list):
        df.loc[i,'wgt'+asset+'_' + model] = wgts[j]
    df.loc[i,'wgt_cal_'+model] = np.sum(wgts)
    df.loc[i,'raver_portret_'+model] = r + wgts @ (df.loc[i,ret_list].values - r ) if np.sum(wgts)<=1 else rb + wgts @ (df.loc[i,ret_list].values -rb)
    df.loc[i,'expret_'+model] = r + wgts @ (means-r) if np.sum(wgts)<=1 else rb + wgts @ (means-rb)
    df.loc[i,'sd'+model] = np.sqrt(wgts @ cov @ wgts)

    # Estimate only standard deviations in covariance matrix
    model = 'est_sds'
    for j, asset in enumerate(asset_list):
        for k in range(j+1,n):
            cov[j, k] = cov[k, j] = 0.0
    wgts = optimal(means, cov, raver, r, rb, short_lb)
    for j, asset in enumerate(asset_list):
        df.loc[i,'wgt'+asset+'_' + model] = wgts[j]
    df.loc[i,'wgt_cal_'+model] = np.sum(wgts)
    df.loc[i,'raver_portret_'+model] = r + wgts @ (df.loc[i,ret_list].values - r ) if np.sum(wgts)<=1 else rb + wgts @ (df.loc[i,ret_list].values -rb)
    df.loc[i,'expret_'+model] = r + wgts @ (means-r) if np.sum(wgts)<=1 else rb + wgts @ (means-rb)
    df.loc[i,'sd'+model] = np.sqrt(wgts @ cov @ wgts)

    # Equal-weighted portfolio
    model = 'ew'
    for j, asset in enumerate(asset_list):
        cov[j,j] = (sds.mean())**2
    wgts = optimal(means, cov, raver, r, rb, short_lb)
    for j, asset in enumerate(asset_list):
        df.loc[i,'wgt'+asset+'_' + model] = wgts[j]    
    # wgts = (1/n)*np.ones(n)
    df.loc[i,'wgt_cal_'+model] = np.sum(wgts)
    df.loc[i,'raver_portret_'+model] = r + wgts @ (df.loc[i,ret_list].values - r ) if np.sum(wgts)<=1 else rb + wgts @ (df.loc[i,ret_list].values -rb)
    df.loc[i,'expret_'+model] = r + wgts @ (means-r) if np.sum(wgts)<=1 else rb + wgts @ (means-rb)
    df.loc[i,'sd'+model] = np.sqrt(wgts @ cov @ wgts)


In [88]:
portret_list = ['raver_portret_' +  model for model in ['est_all', 'est_cov', 'est_sds','ew']]
stats = df[portret_list].describe()
stats

,raver_portret_est_all,raver_portret_est_cov,raver_portret_est_sds,raver_portret_ew
count,34.000000,34.000000,34.000000,34.000000
mean,0.109585,0.112968,0.111712,0.097536
std,0.232138,0.161175,0.150348,0.106861
min,-0.619804,-0.266419,-0.275482,-0.150321
25%,-0.002704,0.049963,0.047921,0.041672
50%,0.151706,0.119686,0.134911,0.100836
75%,0.243888,0.175461,0.200955,0.145017
max,0.571062,0.458854,0.383299,0.303154


In [89]:
# Summarize sharpe ratio, avg ret, sd(ret) for each model
sr_df = pd.DataFrame(dtype=float, columns = ['sr','avg_ret','sd_ret'], index = ['est_all', 'est_cov', 'est_sds','ew'])
r = df[np.isnan(df['raver_portret_ew'])==False].rf.mean()
for model in ['est_all', 'est_cov', 'est_sds','ew']:
    sr_df.loc[model,'sr'] = (stats.loc['mean','raver_portret_' +  model] - r)/stats.loc['std','raver_portret_' +  model]
    sr_df.loc[model,'avg_ret'] = stats.loc['mean','raver_portret_' +  model]
    sr_df.loc[model,'sd_ret'] = stats.loc['std','raver_portret_' +  model]

label_dict = {'true': 'theoretical optimal weights', 
            'ew': 'equal weights',
            'est_all': 'estimate all inputs',
            'est_cov': 'estimate covariance matrix only',
            'est_sds': 'estimate standard deviations only'}

xaxis_label_dict = {'true': 'theoretical optimal weights', 
            'ew': 'Est-None',
            'est_all': 'Est-All',
            'est_cov': 'Est-SD-Corr',
            'est_sds': 'Est-SD'}
sr_df = sr_df.reset_index()
sr_df['label'] = sr_df['index'].apply(lambda x: label_dict[x])
sr_df['xaxis_label'] = sr_df['index'].apply(lambda x: xaxis_label_dict[x])

df[np.isnan(df['raver_portret_ew'])==False].rf
sr_df



,index,sr,avg_ret,sd_ret,label,xaxis_label
0,est_all,0.351030,0.109585,0.232138,estimate all inputs,Est-All
1,est_cov,0.526570,0.112968,0.161175,estimate covariance matrix only,Est-SD-Corr
2,est_sds,0.556138,0.111712,0.150348,estimate standard deviations only,Est-SD
3,ew,0.649799,0.097536,0.106861,equal weights,Est-None


In [90]:
import plotly.graph_objects as go

string ="Strategy: %{customdata[0]} <br>"
string += "Sharpe ratio: %{y:0.3f}<br>"
string += "Average return: %{customdata[1]:0.1%}<br>"
string += "SD(return): %{customdata[2]:0.1%}<br>"
string += "<extra></extra>"

fig = go.Figure()
fig.add_trace(go.Bar(x=sr_df['xaxis_label'], y=sr_df['sr'], customdata=sr_df[['label','avg_ret','sd_ret']], hovertemplate=string))
fig.layout.yaxis["title"] = "Sharpe ratio"
fig.layout.xaxis["title"] = "Strategy"
fig.show()

In [92]:
# Plot the time-series of returns and portfolio weights.
for asset in asset_list:
    df['wgt'+asset + '_ew'] = 1/n
label_dict = {'true': 'theoretical optimal weights', 
            'ew': 'equal weights',
            'est_all': 'estimate all inputs',
            'est_cov': 'estimate covariance matrix only',
            'est_sds': 'estimate standard deviations only'}

xaxis_label_dict = {'true': 'theoretical optimal weights', 
            'ew': 'Est-None',
            'est_all': 'Est-All',
            'est_cov': 'Est-SD-Corr',
            'est_sds': 'Est-SD'}           

fig = go.Figure()
for model in ['est_all', 'est_cov', 'est_sds', 'ew']:
    string =  "Strategy: " + label_dict[model] +" <br>"
    string += "Year: %{x:4.0f}<br>"
    string += "Return: %{y:0.1%}<br>"
    string += "Weight in Risky Portfolio: %{customdata[0]: 0.1%} <br>"
    string += "Risky Portfolio Weights:<br>"
    string += "  "+ risky[0] +": %{customdata[1]: 0.1%} <br>"
    string += "  "+ risky[1] +": %{customdata[2]: 0.1%} <br>"
    string += "  "+ risky[2] +": %{customdata[3]: 0.1%} <br>"
    string += "  "+ risky[3] +": %{customdata[4]: 0.1%} <br>"
    string += "<extra></extra>"

    wgt_list = ['wgt_cal_'+ model] + ['wgt'+asset + "_" + model for asset in asset_list]
    trace=go.Scatter(x=df['year'], y=df['raver_portret_'+model], customdata=df[wgt_list], hovertemplate=string, name = xaxis_label_dict[model])
    fig.add_trace(trace)
fig.layout.yaxis["title"] = "Return"
fig.layout.xaxis["title"] = "Year"
fig.update_yaxes(tickformat=".0%")
fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.01))
fig.update_xaxes(range=[df['year'].iloc[window], df.year.max()])
fig.show()

In [93]:
# Plot leverage (wgt_cal)
fig = go.Figure()
for model in ['est_all', 'est_cov', 'est_sds', 'ew']:
    string =  "Strategy: " + label_dict[model] +" <br>"
    string += "Year: %{x:4.0f}<br>"
    string += "Weight in Risky Portfolio: %{y: 0.1%} <br>"
    string += "<extra></extra>"

    trace=go.Scatter(x=df['year'], y=df['wgt_cal_'+model], hovertemplate=string, name = xaxis_label_dict[model])
    fig.add_trace(trace)
fig.layout.yaxis["title"] = "Risky Portfolio Weight"
fig.layout.xaxis["title"] = "Year"
fig.update_yaxes(tickformat=".0%")
fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.01))
fig.update_xaxes(range=[df['year'].iloc[window], df.year.max()])
fig.show()


In [94]:
# Plot expected return (expret_)
fig = go.Figure()
for model in ['est_all', 'est_cov', 'est_sds', 'ew']:
    string =  "Strategy: " + label_dict[model] +" <br>"
    string += "Year: %{x:4.0f}<br>"
    string += "Expected Return: %{y: 0.1%} <br>"
    string += "<extra></extra>"

    trace=go.Scatter(x=df['year'], y=df['expret_'+model], hovertemplate=string, name = xaxis_label_dict[model])
    fig.add_trace(trace)
fig.layout.yaxis["title"] = "Expected Return"
fig.layout.xaxis["title"] = "Year"
fig.update_yaxes(tickformat=".0%")
fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.01))
fig.update_xaxes(range=[df['year'].iloc[window], df.year.max()])
fig.show()

In [96]:
# Plot standard deviation (sd)
fig = go.Figure()
for model in ['est_all', 'est_cov', 'est_sds', 'ew']:
    string =  "Strategy: " + label_dict[model] +" <br>"
    string += "Year: %{x:4.0f}<br>"
    string += "Expected Standard Deviation: %{y: 0.1%} <br>"
    string += "<extra></extra>"

    trace=go.Scatter(x=df['year'], y=df['sd'+model], hovertemplate=string, name = xaxis_label_dict[model])
    fig.add_trace(trace)
fig.layout.yaxis["title"] = "Standard Deviation (Expected)"
fig.layout.xaxis["title"] = "Year"
fig.update_yaxes(tickformat=".0%")
fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.01))
fig.update_xaxes(range=[df['year'].iloc[window], df.year.max()])
fig.show()

In [97]:
wgt_list = ['wgt_cal_' +  model for model in ['est_all', 'est_cov', 'est_sds','ew']]
stats = df[wgt_list].describe()
stats

,wgt_cal_est_all,wgt_cal_est_cov,wgt_cal_est_sds,wgt_cal_ew
count,34.000000,34.000000,34.000000,34.000000
mean,2.051617,2.068139,2.218264,1.498031
std,0.724315,1.059016,1.033974,0.593711
min,0.999944,1.000000,1.000000,0.755457
25%,1.369769,1.071095,1.238317,1.000000
50%,2.174609,2.071743,2.087902,1.110453
75%,2.682009,2.693867,3.087554,2.111636
max,2.964578,5.271987,3.963956,2.484092
